In [43]:
#подгружаем необходимые библиотеки
import numpy as np
import pandas as pd
import scipy
import category_encoders as ce
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import plotly.express as px
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from joblib import dump

In [2]:
#подгрузим основной датасет и удалим дубликаты
events = pd.read_csv("events.csv")
events = events.drop_duplicates()

In [3]:
#поменяем названия значений в столбце event
ord_encoder = ce.OrdinalEncoder(mapping=[{
	'col': 'event',
	'mapping': {'view': 1, 'addtocart': 2, 'transaction': 3}
}])
# применяем трансформацию к столбцу
data_bin = ord_encoder.fit_transform(events[['event']])
events.drop('event', axis=1 , inplace=True)
# добавляем результат к исходному DataFrame
events = pd.concat([events, data_bin], axis=1)

In [4]:
#избавимся от пропусков в столбце транзакий
events['transactionid'] = events['transactionid'].fillna(0)

Далее расширим датасет новыми переменными

In [5]:
#вычленим дату
events['date'] = pd.to_datetime(events['timestamp'], unit='ms')
#вычленим день
events['data'] = events['date'].dt.date
#вычленим час дня
events['hour'] = events['date'].dt.hour
#вычленим месяц
events['month'] = events['date'].dt.month
#вычленим день недели
events['day_of_week'] = events['date'].dt.day_of_week
#вычленим выходные
events["weekend"] = events['date'].dt.dayofweek > 4
events['weekend'] = np.where((events.weekend =='False'), 0, events.weekend)
events['weekend'] = np.where((events.weekend =='True'), 1, events.weekend)
#создадим столбец с праздничными днями
events['data'] = events['data'].astype(str)
events['data'] = events['data'].replace(to_replace=r'-', value = '', regex=True)
events['data'] = events['data'].astype(int)

In [6]:
#создадим функцию для выявления выходных дней
def holidays(x):
    if x >= 20150101 and x <= 20150110:
        return 1
    elif x == 20150224:
        return 1
    elif x == 20150308:
        return 1
    elif x == 20150501:
        return 1
    elif x == 20150509:
        return 1
    elif x == 20150612:
        return 1
    elif x == 20151231:
        return 1
    else:
        return 0
events['hol'] = events['data'].apply(holidays)

In [7]:
#отсортируем датасет по времени
events.sort_values('timestamp', inplace=True)

In [8]:
#удалим столбец data
events.drop('data', axis=1 , inplace=True)
#удалим дубликаты
events = events.fillna(0)

In [9]:
#посмотрим результат
events.head()

,timestamp,visitorid,itemid,transactionid,event,date,hour,month,day_of_week,weekend,hol
1462974,1430622004384,693516,297662,0.0,2,2015-05-03 03:00:04.384,3,5,6,1,0
1464806,1430622011289,829044,60987,0.0,1,2015-05-03 03:00:11.289,3,5,6,1,0
1463000,1430622013048,652699,252860,0.0,1,2015-05-03 03:00:13.048,3,5,6,1,0
1465287,1430622024154,1125936,33661,0.0,1,2015-05-03 03:00:24.154,3,5,6,1,0
1462955,1430622026228,693516,297662,0.0,1,2015-05-03 03:00:26.228,3,5,6,1,0


In [10]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2755641 entries, 1462974 to 1459312
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      int64         
 1   visitorid      int64         
 2   itemid         int64         
 3   transactionid  float64       
 4   event          int64         
 5   date           datetime64[ns]
 6   hour           int64         
 7   month          int64         
 8   day_of_week    int64         
 9   weekend        int64         
 10  hol            int64         
dtypes: datetime64[ns](1), float64(1), int64(9)
memory usage: 252.3 MB


Проанализируем данные

In [11]:
#посмотрим на количесто уникальных пользователей
len(events['visitorid'].unique())

1407580

In [12]:
#посмотрим на количесто уникальных товаров
len(events['itemid'].unique())

235061

In [ ]:
#посмотрим на значения переменной "event"
fig = px.histogram(
data_frame=events,
x='event',
title='Распределение признака event',
width=1000,
height=500,
histnorm='percent',
)
marginal='box',
fig.show()

Наиболее популярное действие - view. Поэтому рекомендательную систему построить на основании данного показателя

In [14]:
#создадим новый столбец, в котором будут только те строчки, которые означают, что покупатель просматривал данный товар 
events['event_value'] = (events['event'] == 1).astype(int)

In [ ]:
#посмотрим в какое время чаще совершаются просмотры на сайте
event_data = (events.groupby('hour')['event_value'].sum().reset_index())
#строим график
fig = px.bar(
    data_frame=event_data,
    x='hour',
    y='event_value', 
    color='event_value',
    orientation='v',
    height=500,
    width=1000,
    title='Просмотры на сайте по часам дня'
)
fig.show()

Пик действий на сайте приходится на конец дня

In [ ]:
#посмотрим в какой месяц чаще совершаются просмотры на сайте
month_data = (events.groupby('month')['event_value'].sum().reset_index())
#строим график
fig = px.bar(
    data_frame=month_data,
    x='month',
    y='event_value', 
    color='event_value',
    orientation='v',
    height=500,
    width=1000,
    title='Просмотры на сайте по месяцам'
)
fig.show()

Июль - наиболее популярный месяц по просмотрам

In [ ]:
#посмотрим в какой день недели чаще совершаются просмотры на сайте
week_data = (events.groupby('day_of_week')['event_value'].sum().reset_index())
#строим график
fig = px.bar(
    data_frame=week_data,
    x='day_of_week',
    y='event_value', 
    color='event_value',
    orientation='v',
    height=500,
    width=1000,
    title='Просмотры на сайте по дням недели'
)
fig.show()

Начало недели - наиболее продуктивно по просмотрам товаров

In [ ]:
#посмотрим траты на покупки товаров в выходные и будние дни
weekend_data = (events.groupby('weekend')['transactionid'].mean().reset_index())
#строим график
fig = px.bar(
    data_frame=weekend_data,
    x='weekend',
    y='transactionid', 
    color='transactionid',
    orientation='v',
    height=500,
    width=1000,
    title='Покупки на сайте по выходным'
)
fig.show()

Люди тратят деньги чаще также в будние дни

In [ ]:
#посмотрим просмотры товаров в будние и праздничные дни
hol_data = (events.groupby('hol')['transactionid'].mean().reset_index())
#строим график
fig = px.bar(
    data_frame=hol_data,
    x='hol',
    y='transactionid', 
    color='transactionid',
    orientation='v',
    height=500,
    width=1000,
    title='Покупки на сайте по праздничным и обычным дням'
)
fig.show()

Такое же соотношение и по отношению к праздничным дням, в эти дни также тратят меньше

Перейдем к построению рекомендательной системы

In [24]:
#уберем пользователей, редко посещающих интернет-магазин, ограничим это число 30 посещениями
rm = events['visitorid'].value_counts().loc[lambda x : x < 30].index.tolist()
events = events[events.visitorid.isin (rm) == False ]

In [27]:
#разделим выборку на обучающую и тестовую
train, test = train_test_split(events, test_size=0.2, shuffle=False)

In [28]:
#создадим матрицы для построения рекомендательных систем
train_pivot = pd.pivot_table(
    train,
    index="visitorid", 
    columns="itemid", 
    values="event_value"
)
test_pivot = pd.pivot_table(
    test,
    index="visitorid", 
    columns="itemid", 
    values="event_value"
)

print(train_pivot.shape)
print(test_pivot.shape)

(2821, 46337)
(1267, 18618)


In [29]:
#создадим матрицу всего датасета
shell = pd.pivot_table(
    events, 
    index="visitorid", 
    columns="itemid", 
    values="event_value",
    aggfunc=lambda x: 0
)
shell.head()

itemid,6,15,16,17,19,25,26,33,42,54,...,466776,466781,466783,466789,466796,466830,466843,466849,466861,466864
visitorid,,,,,,,,,,,,,,,,,,,,,
172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#соединим матрицы, заполним пропуски, добавим к каждому значению "+ 1" для корректной работы модели
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
print(train_pivot.shape)
print(test_pivot.shape)

train_pivot.head()

(3170, 53393)
(3170, 53393)


itemid,6,15,16,17,19,25,26,33,42,54,...,466776,466781,466783,466789,466796,466830,466843,466849,466861,466864
visitorid,,,,,,,,,,,,,,,,,,,,,
172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
#для работы с моделью построим разреженные матрицы
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)

In [33]:
#построим модель "LightFM" с гиперпараметрами
model_lfm = LightFM(no_components=150, loss='warp', learning_rate=0.05, learning_schedule="adadelta", random_state=42)
model_lfm.fit(train_pivot_sparse, epochs=30, num_threads=2)

In [34]:
#посмотрим метрику
map_at3 = precision_at_k(model_lfm, test_pivot_sparse, k=3).mean()
print('Mean Average Precision at 3: {:.3f}'.format(map_at3))

Mean Average Precision at 3: 0.075


In [36]:
#сохраним модель
#dump(model_lfm, 'model_lfm.joblib')

Поскольку список юзеров в изначальном датасете и список юзеров в новом датасете не совпадают, создадим списки данных пользователей

In [37]:
#создадим список изначальных пользователей
old_users = train_pivot.index.to_list()
#сбросим индексы для построения рекомендательной системы
train_pivot.reset_index(drop=True, inplace=True)
#создадим список новых пользователей
new_users = list(train_pivot.index.values)

In [38]:
#создадим отдельный датасет, который сопоставит старый и новый список пользователей
users_df =  pd.DataFrame(
    {'model_user': new_users,
     'real_user': old_users
    })

In [39]:
users_df.head()

,model_user,real_user
0,0,172
1,1,1014
2,2,1722
3,3,1879
4,4,2019


In [40]:
#создадим функцию для перевода значения из изначального списка в новый список
def rs_unit(x):
    xx = users_df[(users_df["real_user"] == x)]
    number = int(xx["model_user"])
    return number

In [41]:
#создадим уникальные значения юзеров и товаров, для предсказывания товаров с помощью модели
unique_items = np.array(train_pivot.columns)
item_ids = np.array(train_pivot.index)

Последний этап - функция для рекомендации товаров для отдельного пользователя

In [42]:
def units():
    user_id = input('Введите номер пользователя:')
    user_id = int(user_id)
    if user_id in old_users:
        user_id_new = rs_unit(user_id)
        list_pred = model_lfm.predict(user_id_new, item_ids)
        recomendations_ids = np.argsort(-list_pred)[:3]
        recomendations = unique_items[recomendations_ids]
        print('Рекомендации для пользователя {}: {}'.format(user_id, recomendations))
    else:
        print('Такого пользователя не существует. Введите другого пользователя')

for i in range(1):
    units()

Рекомендации для пользователя 172: [27248 10034 18519]
